In [1]:
from mbmvpa.preprocessing.bids import bids_preprocess
from mbmvpa.preprocessing.events import events_preprocess
from mbmvpa.data.loader import prepare_dataset

In [2]:
root = '/data2/project_model_based_fmri/ds000005'

In [4]:
X, voxel_mask, layout = bids_preprocess(root, smoothing_fwhm=None, zoom=(2, 2, 2), ncore=3, nthread=4)


  0%|          | 0/6 [00:00<?, ?it/s]
loading bids dataset..                            :   0%|          | 0/6 [00:00<?, ?it/s]
loading bids dataset..                            :  17%|█▋        | 1/6 [00:09<00:46,  9.28s/it]
making custom voxel mask..                        :  17%|█▋        | 1/6 [00:09<00:46,  9.28s/it]
making custom voxel mask..                        :  33%|███▎      | 2/6 [00:09<00:26,  6.54s/it]
image preprocessing - parameter setting..         :  33%|███▎      | 2/6 [00:09<00:26,  6.54s/it]
image preprocessing - parameter setting..         :  50%|█████     | 3/6 [00:12<00:16,  5.49s/it]
image preprocessing - making path..               :  50%|█████     | 3/6 [00:12<00:16,  5.49s/it]
image preprocessing - fMRI data..                 :  67%|██████▋   | 4/6 [00:12<00:10,  5.49s/it]/home/mybirth0407/anaconda3/envs/mvpa/lib/python3.7/site-packages/scipy/ndimage/interpolation.py:478: UserWarning: The behaviour of affine_transform with a one-dimensional array supplied

In [5]:
def example_tom_adjust_columns(row):
    ## rename data in a row to the name which can match hbayesdm.ra_prospect requirements ##
    row["gamble"] = 1 if row["respcat"] == 1 else 0
    row["cert"] = 0
    return row

In [6]:
def example_tom_condition(row):
    # include all trial data
    return True

In [7]:
def example_tom_modulation(row, param_dict):
    ## calculate subjectives utility for choosing Gamble over Safe option
    ## prospect theory with loss aversion and risk aversion is adopted
    modulation = (row["gain"] ** param_dict["rho"]) - (param_dict["lambda"] * (row["loss"] ** param_dict["rho"]))
    row["modulation"] = modulation
    return row

In [8]:
dm_model, df_events, signals, time_masks, _ = \
    events_preprocess(root,
                      preprocess=example_tom_adjust_columns,
                      condition=example_tom_condition,
                      modulation=example_tom_modulation,
                      dm_model='ra_prospect')


  0%|          | 0/6 [00:00<?, ?it/s]
loading bids dataset..                            :   0%|          | 0/6 [00:00<?, ?it/s]
loading bids dataset..                            :  17%|█▋        | 1/6 [00:18<01:30, 18.15s/it]
processing event file columns..                   :  17%|█▋        | 1/6 [00:18<01:30, 18.15s/it]
processing event file columns..                   :  33%|███▎      | 2/6 [00:27<01:02, 15.54s/it]
calculating time masks..                          :  33%|███▎      | 2/6 [00:27<01:02, 15.54s/it]

NameError: name 'time_length' is not defined

## model

In [ ]:
from model_based_mvpa.models.regressor import *
from model_based_mvpa.utils.coef2map import *
import time

In [ ]:
X, y, voxel_mask = prepare_dataset(root=layout.derivatives["fMRIPrep"].root)

In [ ]:
print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
coefs = mlp_regression_v2(X, y,
                       layer_dims=[1024, 1024],
                       activation_func='linear',
                       dropout_rate=0.5,
                       epochs=100,
                       patience=10,
                       batch_size=64,
                       N=3,
                       verbose=1)

task_name = 'tom2007_mlp_v2'

result = get_map(coefs, voxel_mask, task_name, map_type='z', save=True, save_path='./results' sigma=1)
print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
coefs = penalized_linear_regression(X, y,
                             alpha=0.001,
                             lambda_par=2.0,
                             epochs=100,
                             patience=30,
                             batch_size=256,
                             N=30,
                             verbose=1)

task_name = 'tom2007_penalized_linear'

result = get_map(coefs, voxel_mask, task_name, map_type='z', save=True, save_path='./results' sigma=1)
print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
coefs = penalized_linear_regression(X, y,
                             alpha=0.001,
                             lambda_par=2.0,
                             epochs=100,
                             patience=30,
                             batch_size=256,
                             N=30,
                             verbose=1)

task_name = 'tom2007_penalized_linear'

result = get_map(coefs, voxel_mask, task_name, map_type='z', save=True, save_path='./results' sigma=1)
print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
coefs = elasticnet(X, y,max_use_sample_N=10000,
             alpha=0.001,
             n_jobs=16,
             N=3,
             verbose=3)

task_name = 'tom2007_elasticnet'

result = get_map(coefs, voxel_mask, task_name, map_type='z', save=True, save_path='./results' sigma=1)
print(time.strftime('%c', time.localtime(time.time())))